In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

model_name = os.getenv('OPENAI_MODEL')
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")
model.invoke("Hello, how are you?")

AIMessage(content="Hello! I'm just a program, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 13, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-0ceb9cae-79ff-43e2-809a-8ae903435691-0', usage_metadata={'input_tokens': 13, 'output_tokens': 24, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Best Practices

## Simple LLM Application

In [7]:
# OpenAI's chat completion message, including roles as developers, users, and assistant
# langchain-core provides messages for developers, users, and assistant
# langchain also support OpenAI API's message format

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage( "Hello, how are you?"),
    HumanMessage("hi!"),
]

# model.invoke(messages)

for token in model.stream(messages):
    print(token.content)


Hello
!
 How
 can
 I
 assist
 you
 today
?



In [16]:
# langchain provides prompt template
from langchain_core.prompts import ChatPromptTemplate

sys_template = ChatPromptTemplate.from_messages(
    [
        ("system", "translate the following text into {language}"),
        ("user", "{text}"),
    ]
)

prompt = sys_template.invoke(
    {
        "language": "French",
        "text": "Hello, how are you?",
    }
)

print(prompt)

for token in model.stream(prompt):
    print(token.content)

messages=[SystemMessage(content='translate the following text into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

Bonjour
,
 comment
 ça
 va
 ?



## Chatbot

How to build a conversation?

### 1. Naive way: record messages manually.
```python
from langchain_core.messages import AIMessage, HumanMessage
Messages = [
    HumanMessage("I'm Tom"),
    AIMessage("Hello Tom!"),
    HumanMessage("What's my name?"),
]

# AIMessage(content="Your name is Tom")
```



### 2. memory with langgraph

In [20]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state['messages'])
    return {"messages": response}

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [ ]:

config = {"configurable": {"thread_id": "abc123"}}

query = "Hi. I'm Tom. How are you doing?"

msg_input = [HumanMessage(query)]
response = app.invoke({"messages": msg_input}, config)



In [30]:
query = "What's my name?"
msg_input = [query]
response = app.invoke({"messages": msg_input}, config)
response

{'messages': [HumanMessage(content="Hi. I'm Tom. How are you doing?", additional_kwargs={}, response_metadata={}, id='f7a9b3c7-b2d8-472f-8b04-95b33244bd5d'),
  AIMessage(content="Hi Tom! I'm just a program, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 17, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-23628512-262e-44c4-9eba-7bdd274afae7-0', usage_metadata={'input_tokens': 17, 'output_tokens': 25, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content="

In this way, the conversation is memorized in the checkpoint located in the memory.

## Agent

What is agent?

An agent is a system that uses LLM to make human-like decisions and executions.

